### Практическая работа к уроку №1

In [1]:
import numpy as np
import random

#### Доделать все функции, где стоит комментарий "сделать дома"

In [2]:
recommended_list = [143,156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

prices_recommended = [random.randint(50, 500) for i in range(len(recommended_list))]
print(prices_recommended)

# Если релевантный товар, есть в списке рекомендаций,
# то подставляем для него уже раннее определенную цену.
# Если нет - рандомно
prices_bought = []
for id_bought in bought_list:
    try:
        prices_bought.append(prices_recommended[recommended_list.index(id_bought)])
    except ValueError:
        prices_bought.append(random.randint(50, 500))

print(prices_bought)

[111, 402, 184, 238, 389, 367, 486, 141, 428, 230]
[88, 152, 111, 238]


#### 1. Hit rate 

In [3]:
def hit_rate(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return (flags.sum() > 0) * 1

def hit_rate_at_k(recommended_list, bought_list, k=5):
    return hit_rate(recommended_list[:k], bought_list)

In [4]:
hit_rate(recommended_list, bought_list)

1

In [5]:
hit_rate_at_k(recommended_list, bought_list, k=5)

1

#### 2. Precision

*Precision* - доля релевантных товаров среди рекомендованных = Какой % рекомендованных товаров  юзер купил

Precision= (# of recommended items that are relevant) / (# of recommended items)  

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

Money Precision@k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)  

In [6]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

def money_precision_at_k_(recommended_list, bought_list, prices_recommended, k=5):
    recommend_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.isin(recommend_list, bought_list)
    
    precision = np.dot(flags, prices_recommended).sum() / prices_recommended.sum()
    
    return precision

In [7]:
precision(recommended_list, bought_list)

0.2

In [8]:
precision_at_k(recommended_list, bought_list, k=5)

0.4

In [9]:
money_precision_at_k_(recommended_list, bought_list, prices_recommended, k=5)

0.263595166163142

#### 3. Recall

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

Recall= (# of recommended items that are relevant) / (# of relevant items)  

Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)

In [10]:
def recall(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    return flags.sum() / len(bought_list)

def recall_at_k(recommended_list, bought_list, k=5):
    return recall(recommended_list[:k], bought_list)


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    recommend_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.isin(recommend_list, bought_list)
    
    precision = np.dot(flags, prices_recommended).sum() / np.sum(prices_bought)
    
    return precision

In [11]:
recall(recommended_list, bought_list)

0.5

In [12]:
recall_at_k(recommended_list, bought_list, k=10)

0.5

In [13]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5)

0.5925297113752123

#### 4. AP@k
AP@k - average precision at k

$$AP@k = \frac{1}{r} \sum{[recommended_{relevant_i}] * precision@k}$$

- r - кол-во релевантный среди рекомендованных
- Суммируем по всем релевантным товарам
- Зависит от порядка рекомендаций

In [14]:
def ap_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    
    return np.mean([precision_at_k(recommended_list, bought_list, k=index_relevant + 1)
                    for index_relevant in relevant_indexes])

In [15]:
ap_k(recommended_list, bought_list, k=5)

0.75

#### 5. MAP@k

MAP@k (Mean Average Precision@k)  
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций

$$MAP@k = \frac{1}{|U|} \sum_u{AP_k}$$
  
|U| - кол-во юзеров

In [16]:
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                            [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                            [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                    ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1,2] # юзер 3
                      ] 

In [17]:
def map_k(recommended_list, bought_list, k=5):
    result = [ap_k(recommended, bought, k) 
              for bought, recommended in zip(bought_list_3_users, recommended_list_3_users)]
    
    return np.mean(result)

In [18]:
map_k(recommended_list, bought_list, k=5)

0.3333333333333333

#### 6. Normalized discounted cumulative gain ( NDCG@k)

$$DCG = \frac{1}{|r|} \sum_u{\frac{[bought fact]}{discount(i)}}$$  

$discount(i) = i$ if $i <= 2$,   
$discount(i) = log_2(i+1)$ if $i > 2$


(!) Считаем для первых k рекоммендаций   
(!) - существуют вариации с другими $discount(i)$  
i - ранк рекомендованного товара  
|r| - кол-во рекомендованных товаров 

$$NDCG = \frac{DCG}{ideal DCG}$$


$DCG@5 = \frac{1}{5}*(1 / 1 + 0 / 2 + 0 / log(3) + 1 / log(4) + 0 / log(5))$  
$ideal DCG@5 = \frac{1}{5}*(1 / 1 + 1 / 2 + 1 / log(3) + 1 / log(4) + 1 / log(5))$  

$NDCG = \frac{DCG}{ideal DCG}$

In [19]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    recommend_list = np.array(recommended_list)[:k]
    len_rec = len(recommend_list)
    
    flags = np.isin(recommend_list, bought_list).astype('int')
    
    discount = [i if i <= 2 else np.log2(i+1) for i in range(1, len_rec + 1)]
    
    dcg = np.sum(flags / discount) / len_rec
    
    ideal_dcg = np.sum(np.ones(len_rec) / discount) / len_rec
    
    return dcg / ideal_dcg

In [20]:
ndcg_at_k(recommended_list, bought_list, k=5)

0.5077769820926182

#### 7. Mean Reciprocal Rank ( MRR@k )

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $k_u$
- Посчитать reciprocal rank = $\frac{1}{k_u}$

$$MRR = mean(\frac{1}{k_u})$$

In [21]:
def reciprocal_rank(recommended_list, bought_list, k=1):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    
    if len(relevant_indexes) == 0:
        return 0
    
    return 1 / (relevant_indexes[0] + 1)

def reciprocal_rank_users(recommended_list_users, bought_list_users, k=1):
    result = [reciprocal_rank(recommended, bought, k) 
              for bought, recommended in zip(bought_list_3_users, recommended_list_3_users)]
    
    return np.mean(result)

In [22]:
reciprocal_rank(recommended_list, bought_list, k=5)

1.0

In [23]:
reciprocal_rank_users(recommended_list_3_users, bought_list_3_users, k=5)

0.3333333333333333